*This notebook can be found on* [github](https://github.com/qojulia/QuantumOptics.jl-examples/tree/master/notebooks/wavepacket2D.ipynb)

# Dynamics of a two-dimensional wavepacket hitting a Gaussian potential

In [ ]:
using QuantumOptics, PyPlot

In this example we will show how one can evolve a wavepacket in 2 spatial dimensions. We will do that using [`tensor`](@ref) products between the two spaces. We start similarly to the 1D case, by defining a position basis and a momentum operator for each dimension.

In [ ]:
Npoints = 100
Npointsy = 80

xmin = -30
xmax = 50
b_position = PositionBasis(xmin, xmax, Npoints)
b_momentum = MomentumBasis(b_position)

ymin = -20
ymax = 20
b_positiony = PositionBasis(ymin, ymax, Npointsy)
b_momentumy = MomentumBasis(b_positiony);

The collective `FFTOperator` is defined analogously to the 1D case using the composite bases.

In [ ]:
b_comp_x = b_position ⊗ b_positiony
b_comp_p = b_momentum ⊗ b_momentumy

Txp = transform(b_comp_x, b_comp_p)
Tpx = transform(b_comp_p, b_comp_x);

Thanks to these operators, we can specify the momentum operators in the respective `MomentumBasis`, where they are diagonal. Applying a diagonal operator is of course much more efficient.

In [ ]:
px = momentum(b_momentum)
py = momentum(b_momentumy);

Now that we have a composite basis, we can write each kinetic energy term in this composite basis. In order to keep the `FFTOperator` approach efficient, we will do this using lazy operations.

In [ ]:
Hkinx = LazyTensor(b_comp_p, [1, 2], [px^2/2, one(b_momentumy)])
Hkiny = LazyTensor(b_comp_p, [1, 2], [one(b_momentum), py^2/2])

Hkinx_FFT = LazyProduct(Txp, Hkinx, Tpx)
Hkiny_FFT = LazyProduct(Txp, Hkiny, Tpx);

Now we will add a two-dimensional potential. If we wanted to add a one-dimensional potential $V(x)$ the process is identical to the 1D case.

However, making a 2D potential is a bit more involved. The first thing you need to understand is that all that `potentialoperator` does is create a `diagonaloperator` (since the potential function is always a diagonal operator in the position basis). So, considering the fact that the operators x and y are diagonal in a position basis (and hence also their tensor product), applying a function to a tensor product x ⊗ y results in a diagonal operator where the function is applied to each element of a diagonal.

This is easily done like so

In [ ]:
xsample = samplepoints(b_position)
ysample = samplepoints(b_positiony)

potential(x,y) = exp(-(x^2 + y^2)/30.0);

V = diagonaloperator(b_comp_x, [potential(x, y) for y in ysample for x in xsample]);

Then one creates the full Hamiltonian simply by combining the kinetic and potential terms

In [ ]:
H = LazySum(Hkinx_FFT, Hkiny_FFT, V);

Now we will create a wavepacket in 2D and evolve it.

In [ ]:
x0 = -10
y0 = -5
p0_x = 1.5
p0_y = 0.5
σ = 2.0

ψx = gaussianstate(b_position, x0, p0_x, σ)
ψy = gaussianstate(b_positiony, y0, p0_y, σ)
ψ = ψx ⊗ ψy

T = collect(0.0:0.1:20.0)
tout, ψt = timeevolution.schroedinger(T, ψ, H);

In [ ]:
density = [reshape(abs2.(ψ.data), (Npoints, Npointsy))' for ψ=ψt]
V_plot = reshape(real.(diag(V.data)), (Npoints, Npointsy))'

figure(figsize=(12, 10))
subplot(221)
contourf(xsample, ysample, density[1], cmap="hot")
contourf(xsample, ysample, V_plot, alpha=0.3, cmap="Greys")
annotate(xy=[25, 10], s="t=$(T[1])", fontsize=20)

subplot(222)
contourf(xsample, ysample, density[50], cmap="hot")
contourf(xsample, ysample, V_plot, alpha=0.3, cmap="Greys")
annotate(xy=[25, 10], s="t=$(T[50])", fontsize=20)

subplot(223)
contourf(xsample, ysample, density[80], cmap="hot")
contourf(xsample, ysample, V_plot, alpha=0.3, cmap="Greys")
annotate(xy=[25, 10], s="t=$(T[80])", fontsize=20)

subplot(224)
contourf(xsample, ysample, density[150], cmap="hot")
contourf(xsample, ysample, V_plot, alpha=0.3, cmap="Greys")
annotate(xy=[25, 10], s="t=$(T[150])", fontsize=20)

tight_layout();